## **1.Analytical Deep Dives**

### **A. Funnel Conversion Analysis by Company Size**

In [1]:
# Setup Code (from your previous message, assuming .env is loaded)
import pandas as pd
from sqlalchemy import create_engine, text
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from scipy.stats import chi2_contingency
from dotenv import load_dotenv
import os

load_dotenv()

# Database Connection Details
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')

connection_string = f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}'
engine = create_engine(connection_string)

def run_sql(query):
    with engine.connect() as conn:
        df = pd.read_sql(text(query), conn)
    return df

# Growth Funnel Mart structure 
funnel_query= """
SELECT
    company_size,
    SUM(total_leads) AS TotalLeads,
    SUM(total_customers) AS TotalCustomers,
    SUM(total_revenue) AS TotalRevenue,
    (SUM(total_customers) * 100.0) / NULLIF(SUM(total_leads), 0) AS ConversionRate
FROM
    growth_funnel_mart
GROUP BY 1
ORDER BY ConversionRate DESC;
"""

funnel_by_size_df = run_sql(funnel_query)
print("--- Funnel Metrics by Company Size ---")
print(funnel_by_size_df)

--- Funnel Metrics by Company Size ---
  company_size  TotalLeads  TotalCustomers  TotalRevenue  ConversionRate
0   Enterprise        39.0            14.0     128458.52        35.89744
1   Mid-Market        51.0             9.0      95725.42        17.64706
2          SMB       113.0            15.0     123603.47        13.27434


### **Funnel Conversion Analysis: Key Insights & Actions**

**Key Insight**                                                                                                                                            
The sales funnel exhibits a dual growth mandate: prioritize Mid-Market for value and Enterprise for efficiency. Mid-Market has the highest Average Deal Value (ADV) at $10,636, but its conversion rate (17.65%) is less than half the highly efficient Enterprise rate (35.90%). The high-volume SMB segment is the least efficient (13.27% conversion), indicating significant waste in sales resources.

**Recommended Actions**
1. **Optimize Mid-Market Funnel (High Priority):**

- **Action:** Immediately launch a deep-dive analysis focusing on the Mid-Market SQL-to-Won stage. Every percentage point of lift here unlocks the highest potential revenue due to the superior ADV.

2. **Re-evaluate SMB Lead Strategy (High Priority):**

- **Action:** Redefine qualification criteria for SMBs. Pivot these leads towards a low-touch, self-service product track or a freemium model to reduce sales cycle costs and improve the effective conversion rate.

3. **Scale Enterprise Lead Generation (Medium Priority):**

- **Action:** Increase investment in channels proven to deliver high-quality Enterprise leads. The funnel's current efficiency (35.90%) can handle higher volume, guaranteeing a strong ROI.

### **B. Product Utilization Insights**

In [2]:
# SQL Query: Find the most popular days and compare utilization by size
utilization_query = """
SELECT
    CASE day_of_week
        WHEN 1 THEN 'Sunday'
        WHEN 2 THEN 'Monday'
        WHEN 3 THEN 'Tuesday'
        WHEN 4 THEN 'Wednesday'
        WHEN 5 THEN 'Thursday'
        WHEN 6 THEN 'Friday'
        WHEN 7 THEN 'Saturday'
    END AS DayOfWeek,
    company_size,
    SUM(total_desk_bookings) AS TotalDeskBookings,
    SUM(total_room_bookings) AS TotalRoomBookings
FROM
    office_utilization_mart
GROUP BY 1, 2
ORDER BY TotalDeskBookings DESC;
"""

utilization_df = run_sql(utilization_query)

print("\n--- Office Utilization by Day of Week and Company Size ---")
print(utilization_df.head(10))


--- Office Utilization by Day of Week and Company Size ---
   DayOfWeek company_size  TotalDeskBookings  TotalRoomBookings
0   Thursday          SMB              108.0               84.0
1     Sunday   Enterprise              105.0               83.0
2    Tuesday          SMB              101.0               75.0
3  Wednesday          SMB               99.0               93.0
4   Saturday          SMB               93.0               79.0
5   Thursday   Enterprise               91.0               83.0
6     Friday   Enterprise               91.0               95.0
7   Saturday   Enterprise               88.0               95.0
8     Friday          SMB               87.0               81.0
9     Monday          SMB               87.0              104.0


### **Product Utilization Insights: Key Insights & Actions**

### **Key Insight**
Utilization patterns reveal non-traditional office usage, suggesting client flexibility. Enterprise clients show high utilization on non-standard days (e.g., Sunday and Saturday are among the top 8 days for bookings). More critically, the SMB segment exhibits periods where room bookings exceed desk bookings (e.g., Monday: 104 room bookings vs. 87 desk bookings). This points to an acute need for collaborative spaces over individual work stations during certain days.

### **Recommended Actions**
1. **Develop Utilization Advisory Service (High Priority):**

- **Action:** Use this non-traditional usage data (weekend peaks, high room ratio) to create a personalized consultation service for clients. For example, advise Enterprise clients on managing weekend security/utility costs, and advise SMBs on converting existing office space into more modular, reservable meeting rooms.

2. **Optimize Product for Collaboration (Product Team):**

- **Action:** Investigate the underlying reasons for the high room-to-desk ratio in SMBs. Introduce product features that facilitate room discovery, booking, and check-in efficiency to reduce friction for collaboration-heavy days like Monday.

3. **Use Data as a Sales Differentiator (Sales Team):**

- **Action:** Leverage the unusual usage patterns as evidence that the Deskbird platform provides superior flexibility for global or modern hybrid teams, directly addressing competitors who assume a rigid Tuesday-Wednesday peak.

### **2.Propensity Modeling (Logistic Regression)**

#### **A. Data Preparation**

In [3]:
# Load required data for modeling (Full Funnel Mart + CRM Status)
model_query = """
SELECT
    ffe.user_id,
    ffe.company_size,
    ffe.is_conversion_event,
    CASE WHEN ffe.deal_stage = 'Won' THEN 1 ELSE 0 END AS is_customer
FROM
    int_full_funnel_events AS ffe
WHERE
    ffe.is_conversion_event = 1 -- Only analyze leads that have requested a demo
"""

model_df = run_sql(model_query)

# 1. Clean up the source of the issue: Ensure the company_size column has no lingering issues
#    And ensure the is_customer column is strictly 0 or 1
model_df['is_customer'] = model_df['is_customer'].astype(int)

# Handle the categorical variable (company_size) using one-hot encoding
model_df = pd.get_dummies(model_df, columns=['company_size'], prefix='size', drop_first=True)

# Select features and target
X = model_df.filter(regex='size_') 
y = model_df['is_customer']

# 2. **CRITICAL FIX:** Drop any rows with NaN/None values, which can occur if the join failed 
#    and created a missing category that wasn't fully cleaned up.
#    Since the features are just dummies, this usually means a failure in the initial SQL load.
data = pd.concat([X, y], axis=1).dropna()
X = data.drop(columns=['is_customer'])
y = data['is_customer']

# 3. **CRITICAL FIX:** Final casting to float/int to prevent 'object' type error
X = X.astype(float)
y = y.astype(int)

# Split data (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Add constant for statsmodels (required for the intercept term)
X_train_sm = sm.add_constant(X_train, prepend=False)

#### **B. Build and Analyze Model**

In [4]:
# Build and train the Logistic Regression model
log_reg = sm.Logit(y_train, X_train_sm).fit()

print("\n--- Logistic Regression Model Summary (Propensity to Convert) ---")
print(log_reg.summary())

# Interpretation: Extract coefficients to explain impact using Odds Ratios (exp(coefficient))
results = pd.DataFrame({
    'Feature': log_reg.params.index,
    'Coefficient': log_reg.params.values,
    'Odds Ratio': np.exp(log_reg.params.values)
})

# Filter out constant and present the interpretation
odds_ratios = results[results['Feature'] != 'const'].sort_values(by='Odds Ratio', ascending=False)
print("\n--- Key Predictors (Odds Ratios) ---")
print(odds_ratios)


Optimization terminated successfully.
         Current function value: 0.462336
         Iterations 6

--- Logistic Regression Model Summary (Propensity to Convert) ---
                           Logit Regression Results                           
Dep. Variable:            is_customer   No. Observations:                  142
Model:                          Logit   Df Residuals:                      139
Method:                           MLE   Df Model:                            2
Date:                Thu, 25 Sep 2025   Pseudo R-squ.:                 0.06876
Time:                        16:16:28   Log-Likelihood:                -65.652
converged:                       True   LL-Null:                       -70.499
Covariance Type:            nonrobust   LLR p-value:                  0.007849
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
size_Mid-Market    -1.2840     

#### Propensity Modeling: Key Insights & Actions
**Key Insight**
The Logistic Regression model, which predicts the likelihood of conversion, statistically confirms that the Enterprise segment (the baseline) has the highest propensity to convert among all segments. The odds of a Mid-Market lead converting are 72.3% lower (Odds Ratio of 0.277), and the odds of an SMB lead converting are 81.1% lower (Odds Ratio of 0.189) compared to Enterprise. This contrasts with the Funnel Analysis, suggesting high-value Mid-Market leads are extremely hard to close.

**Recommended Actions**
1. **Lead Scoring Overhaul (High Priority):**

- **Action:** Immediately implement a lead scoring system that heavily weights the Enterprise segment. Sales teams should dedicate disproportionate time and resources to Enterprise leads, as their probability of conversion is the highest.

2. **Diagnostic for Mid-Market (High Priority):**

- **Action:** Conduct a deep diagnostic of the Mid-Market segment's sales cycle to understand why their closing probability is so low. This is the most valuable lead if closed, but the model shows the current process is failing. Investigate common deal stages where they are "Lost."

3. **De-prioritize Low-Value/Low-Propensity:**

- **Action:** Automate follow-up for the SMB segment and shift them entirely to a less expensive, low-touch or automated marketing nurture stream, diverting sales resources to the high-propensity Enterprise segment.

### **3. A/B Testing Analysis (Based on Chi-Squared P-Value)**

In [6]:
conversions_control = 120
visitors_control = 5000
conversions_treatment = 165
visitors_treatment = 5000

# Create a contingency table for the Chi-Squared test
contingency_table = pd.DataFrame({
    'Control': [conversions_control, visitors_control - conversions_control],
    'Treatment': [conversions_treatment, visitors_treatment - conversions_treatment]
}, index=['Conversions', 'No Conversions'])

print("A/B Test Contingency Table ")
print(contingency_table)

# Perform the Chi-Squared test
chi2, p_value, _, _ = chi2_contingency(contingency_table)

alpha = 0.05

print(f"\nChi-Squared Test Results:")
print(f"Chi-Squared Statistic: {chi2:.3f}")
print(f"P-Value: {p_value:.5f}")
print(f"Significance Level (alpha): {alpha}")

--- A/B Test Contingency Table ---
                Control  Treatment
Conversions         120        165
No Conversions     4880       4835

Chi-Squared Test Results:
Chi-Squared Statistic: 6.992
P-Value: 0.00819
Significance Level (alpha): 0.05


### **A/B Testing Analysis: Key Insights & Actions***
**Key Insight**
The test, comparing the standard pricing page (Control) to the new free trial offer (Treatment), resulted in a statistically significant lift in demo request conversions.

- **Conversion Rate (Control):** 120/5,000=2.40%

- **Conversion Rate (Treatment):** 165/5,000=3.30%

- **Lift:** The Treatment delivered a 37.5% increase in conversion.

- **Significance:** The P-Value of 0.00819 is well below the α=0.05 threshold, confirming the uplift is reliable and not due to random chance.

**Recommended Actions**
1. **Immediate Full Rollout (High Priority):**

- **Action:** Immediately transition 100% of website traffic to the new pricing page (Treatment). This change is proven to increase the top-of-funnel lead volume without added advertising cost.

2. **Verify Downstream Funnel Health (Post-Rollout):**

- **Action:** Monitor the conversion rate of these new, higher-volume leads through the entire sales funnel (Lead → SQL → Won). Ensure the increase in volume does not dilute the high conversion efficiency established for the Enterprise segment.

3. **Iterate on the Offer (Marketing Team):**

- **Action:** Use the free trial success as a template. Launch new A/B tests to optimize elements like the duration of the trial, specific features included, and placement of the call-to-action to find further incremental conversion gains.